In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import zipfile
# train_zip_file = '../input/dogs-vs-cats-redux-kernels-edition/train.zip'
# test_zip_file = '../input/dogs-vs-cats-redux-kernels-edition/test.zip'
# with zipfile.ZipFile(train_zip_file,"r") as z:
#     z.extractall(".")
# with zipfile.ZipFile(test_zip_file,"r") as z:
#     z.extractall(".")

In [ ]:
# ! pip install loguru

In [ ]:
! ls /kaggle/working

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import pandas as pd
from loguru import logger

logger.remove()
logger.add("./logs/file_{time}.log", )

train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

class DogCat(Dataset):
    def __init__(self, root, train=True, test=False, transform=None):
        self.root = root
        self.train = train
        self.test = test
        self.transform = transform
        images = [ os.path.join(root, image) for image in os.listdir(root)]
        if self.test:
            self.images = images
        elif train:
            images_len = len(images)
            self.images = images[:int(0.8*images_len)]
        else:
            images_len = len(images)
            self.images = images[int(0.8*images_len):]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        data = Image.open(image_path)
        if self.test:
            return self.transform(data), image_path.split('.')[-2].split('/')[-1]
        if 'cat' in image_path:
            label = 0
        else:
            label = 1
        return self.transform(data), label

data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
train_set = DogCat(train_dir, transform=data_transform)
valid_set = DogCat(train_dir, train=False, transform=data_transform)
test_set = DogCat(test_dir, train=False, test=True, transform=data_transform)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=256, shuffle=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=True)
# inputs, labels = iter(train_loader).next()
# fig = plt.figure(figsize=(24, 16))
# fig.tight_layout()
# for idx in range(24):
#     plt.subplot(4,6, idx+1)
#     plt.title('dog' if labels[idx] == 1 else 'cat')
#     plt.axis('off')
#     # np.transpose()  returns the modified array.
#     plt.imshow(np.transpose(inputs[idx], (1,2,0)))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet_18 = torchvision.models.resnet18(num_classes=2)
resnet_18.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_18.parameters(), lr=0.001, momentum=0.9)

max_epoch = 64
for epoch in range(max_epoch):
    for idx, (inputs,labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        logger.debug('epoch: {}, batch: {}, loss: {}'.format(epoch+1, idx+1, loss.data))
torch.save(resnet_18.state_dict(), 'resnet_18.pth')

resnet_18 = torchvision.models.resnet18(num_classes=2)
resnet_18.load_state_dict(torch.load('resnet_18.pth'))

# inputs, labels = iter(valid_loader).next()
# outputs = resnet_18(inputs)
# _, predicted = torch.max(outputs, 1)

# fig = plt.figure(figsize=(24, 16))
# fig.tight_layout()
# for idx in range(24):
#     plt.subplot(4,6, idx+1)
#     plt.title('dog' if predicted[idx] == 1 else 'cat')
#     plt.axis('off')
#     # np.transpose()  returns the modified array.
#     plt.imshow(np.transpose(inputs[idx], (1,2,0)))

# test
submission = pd.DataFrame(columns=('id', 'label'))
for inputs, image_id in test_loader:
    outputs = resnet_18(inputs)
    _, predicted = torch.max(outputs, 1)
    for idx in range(predicted.size(0)):
        submission = submission.append([{'id': image_id[idx], 'label': predicted[idx].data}])
submission.to_csv('submission.csv', index=False)

# valid
with torch.no_grad():
    total = 0
    correct = 0
    for idx, data in enumerate(valid_loader):
        images, labels = data
        outputs = resnet_18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        batch_correct = (predicted == labels).sum().item()
        correct += batch_correct
        logger.debug('batch:{}, accuracy:{}% '.format(idx+1, 100 * batch_correct / 256))
    logger.debug('total:{}, correct:{}, total accuracy:{}% '.format(total, correct, correct*100 / total))